In [1]:
#Install all dependency packages for the course
#Remember to execute this before running any of the exercises
!pip install tenacity==9.0.0
!pip install langchain==0.3.12
!pip install langchain-openai==0.2.12
!pip install langchain_community==0.3.12
!pip install langgraph==0.2.59
!pip install pysqlite3-binary==0.5.4
!pip install langchain_chroma==0.1.4
!pip install pandas==2.2.3
!pip install pypdf==5.1.0
!pip install nbformat==5.10.4

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     ---------------------------------------- 7.8/7.8 MB 48.8 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)


ERROR: Could not find a version that satisfies the requirement pysqlite3-binary==0.5.4 (from versions: none)
ERROR: No matching distribution found for pysqlite3-binary==0.5.4


     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     ---------------------------------- ---- 13.9/15.8 MB 74.9 MB/s eta 0:00:01
     ---------------------------------------- 15.8/15.8 MB 68.9 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing met

ERROR: Cannot install langchain_chroma because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


     ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
     ---------------------------------------- 4.4/4.4 MB 38.0 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): still running...
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Created wheel for pandas: filename=pandas-2.2.3-cp313-cp313-win_arm64.whl size=37865045 sha256=6c5c12b21607a71349842fa27f54d7bce70f3c529e615b51e16d210b03a37d65
  Stored in directory: c:\users\giris\appdata\local\pip\cache\wheels\98\55

## 2.3. Setup Function Tools for ReAct Agent

In [3]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y


## 2.4. Create a basic ReAct Agent

In [4]:
from langchain_openai import AzureChatOpenAI
import os
#Setup the LLM for the agent

#API info. Replace with your own keys and end points
os.environ["AZURE_OPENAI_API_KEY"]="4e4ab31800a64ae892cbb768fe28c0fc"
os.environ["AZURE_OPENAI_ENDPOINT"]="https://agentic-ai-course-kumaran.openai.azure.com/"

#Setup the LLM
model = AzureChatOpenAI(
    azure_deployment="gpt-4o" ,
    api_version="2023-03-15-preview",
    model="gpt-4o"
)

#Test the model
#response = llm.invoke("Hello, how are you?")
#print(response.content)


In [5]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available. 
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools)


## 2.5. Execute the ReAct Agent

In [6]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The sum of 2 and 3 is 5. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (call_G4DhTdCh4rKR4TAWkfVn7E0b)
 Call ID: call_G4DhTdCh4rKR4TAWkfVn7E0b
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [7]:
#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : 3 multiplied by 2 is 6, and 5 + 1 is also 6. 

Step by Step execution : 
================================ Human Message =================================

What is 3 multipled by 2 and 5 + 1 ?
================================== Ai Message ==================================
Tool Calls:
  find_product (call_nlwakLV1BZaIgMUHBW7phBPj)
 Call ID: call_nlwakLV1BZaIgMUHBW7phBPj
  Args:
    x: 3
    y: 2
  find_sum (call_Pz14Bpqzsv63UdQK7znMElqO)
 Call ID: call_Pz14Bpqzsv63UdQK7znMElqO
  Args:
    x: 5
    y: 1
================================= Tool Message =================================
Name: find_product

6
================================= Tool Message =================================
Name: find_sum

6
================================== Ai Message ==================================

3 multiplied by 2 is 6, and 5 + 1 is also 6.


## 2.6. Debugging the Agent

In [8]:
agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'what is the sum of 2 and 3 ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is the sum of 2 and 3 ?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='a4364fc8-0afc-4f46-971a-ea1acff7f3a1')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='a4364fc8-0afc-4f46-971a-ea1acff7f3a1')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sLAPT3bdzKiVm0Qy6jiQ86z6', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}